# Wine Price Prediction - Extract, Transform, Load (ETL)

## Data and Use case

[**Wine Reviews** - 130k wine reviews with variety, location, winery, price, and description](https://www.kaggle.com/zynicide/wine-reviews/home)

This dataset is available on Kaggle and contains around 130k of wine reviews. The data was scraped from [WineEnthusiast](http://www.winemag.com/?s=&drink_type=wine) on November 22nd, 2017.

I plan to use this dataset to develop a model that predicts wine price for specified set of parameters, like wine variety, region, desired quality. Such model, may be integrated into an application that runs on a mobile device to suggest price range during wine shopping without need to do online search.

## Extract, Transform, Load (ETL)

This task is an important step in transforming the data from the source system into a shape
suitable for analytics. In traditional data warehousing this incudes accessing the OLTP
system’s databases, transforming from a highly normalized data model into a star of
snowflake scheme, and finally storing data to a data warehouse. 

In our project this step is much simpler: data arrives already in CSV format. Finally, the result ends up in
the Cloud Object Storage.

### Load Source Data
Let's load the source dataset from the CSV file we got from Kaggle.

In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
from io import StringIO
import requests
import json
import pandas as pd

# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file(container, filename, credentials):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['name'],'domain': {'id': credentials['domain']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_1 = pd.read_csv(get_object_storage_file('CourseraIoTProject', 'winemag-data-130k-v2.csv', credentials))
df_data_1.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


Column descriptions:
- country - The country that the wine is from
- description - A few sentences from a sommelier describing the wine's taste, smell, look, feel, etc.
- designation - The vineyard within the winery where the grapes that made the wine are from
- points - The number of points WineEnthusiast rated the wine on a scale of 1-100 (though they say they only post reviews for wines that score >=80)
- price - The cost for a bottle of the wine
- province - The province or state that the wine is from
- region_1 - The wine growing area in a province or state (ie Napa)
- region_2 - Sometimes there are more specific regions specified within a wine growing area (ie Rutherford inside the Napa Valley), but this value can sometimes be blank
- taster_name - Name of the person who tasted and reviewed the wine
- taster_twitter_handle - Twitter handle for the person who tasted and reviewed the wine
- title - The title of the wine review, which often contains the vintage if you're interested in extracting that feature
- variety - The type of grapes used to make the wine (ie Pinot Noir)
- winery - The winery that made the wine

### Data Transformation

In [3]:
import matplotlib.pyplot as plt # Library to plot graphics
import numpy as np # To handle with matrices
import seaborn as sns # to build modern graphics

import warnings
warnings.filterwarnings('ignore')

Let's see data format, size, shape, and number of null values:

In [4]:
print(df_data_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
Unnamed: 0               129971 non-null int64
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(2), object(11)
memory usage: 13.9+ MB
None


**Add The Vintage Year**  

The title of the wine review often contains the vintage. Let's check number of records that contain years 19xx-20xx in the title.

In [5]:
year_pattern = ".*?(19[0-9]{2}|20[0-9]{2})"

print("Total number of records:", df_data_1.shape[0])
print("Number of records that contain a year:", df_data_1.title.str.contains(year_pattern).sum())

Total number of records: 129971
Number of records that contain a year: 125345


Looks like we managed to extract the vintage for quite big part of the records. Let's check what was extracted for the first rows in the dataset.

In [6]:
df_data_1[['title']].head(5).title.str.extract(year_pattern)

0    2013
1    2011
2    2013
3    2013
4    2012
Name: title, dtype: object

Let's try to process the whole dataset and see some statistics on the vintage.

In [7]:
import re

def extract_year(title):
    years = [int(s) for s in re.findall(year_pattern, title)]
    return max(years) if len(years) > 0 else None

In [8]:
df_data_1['year'] = list(map(extract_year, df_data_1['title']))
df_data_1[['year']].describe()

,year
count,125345.000000
mean,2010.681998
std,3.675362
min,1904.000000
25%,2009.000000
50%,2011.000000
75%,2013.000000
max,2067.000000


Values distribution and the maximal value look quite logical. Let's check the lower bound.

In [9]:
df_data_1[df_data_1.year < 1950].head(5)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
41441,41441,Portugal,"This was a great vintage Port year, and this w...",Colheita White,94,980.0,Port,NaN,NaN,Roger Voss,@vossroger,Kopke 1935 Colheita White (Port),Port,Kopke,1935.0
51898,51898,Spain,This gorgeous dessert wine shows a coffee-colo...,Don PX Reserva Especial,93,258.0,Andalucia,Montilla-Moriles,NaN,Michael Schachner,@wineschach,"Bodegas Toro Albala, SL 1947 Don PX Reserva Es...",Pedro Ximénez,"Bodegas Toro Albala, SL",1947.0
54748,54748,Spain,"A rank well above most PX sweeties, with lush ...",Solera 1927,94,25.0,Andalucia,Montilla-Moriles,NaN,Michael Schachner,@wineschach,Alvear NV Solera 1927 Pedro Ximénez (Montilla-...,Pedro Ximénez,Alvear,1927.0
56426,56426,France,Exquisite lightness and weightless charm are t...,1904 Cuvée Spéciale Brut,91,40.0,Alsace,Crémant d'Alsace,NaN,Anne Krebiehl MW,@AnneInVino,Arthur Metz NV 1904 Cuvée Spéciale Brut Sparkl...,Sparkling Blend,Arthur Metz,1904.0
74959,74959,Spain,"Apple, lemon-lime and bready aromas are welcom...",1919 Brut Selecció,88,13.0,Catalonia,Cava,NaN,Michael Schachner,@wineschach,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),Sparkling Blend,L'Arboc,1919.0


**Remove Duplicates**  
During the exploration we found, that the datset contains duplicated reviews. Let's drop the duplicates.

In [10]:
df_data_1.drop_duplicates(subset='description', inplace=True)
print ('Number of description duplicates:', df_data_1[df_data_1.duplicated('description',keep=False)].shape[0])

Number of description duplicates: 0


**Drop Unnesecary Columns**  

Let's drop columns we are not going to use for our model.

In [11]:
df_data_1 = df_data_1[['country', 'points', 'price', 'province', 'variety', 'year']]

In [12]:
df_data_1.head(1)

,country,points,price,province,variety,year
0,Italy,87,NaN,Sicily & Sardinia,White Blend,2013.0


## Save Results

Let's save the transformed data frame into Cloud Object Storage as a CSV file.

In [13]:
def put_file(container, local_file_name, credentials): 
    """This functions returns a StringIO object containing the file content from Bluemix Object Storage V3.""" 
    f = open(local_file_name,'rb') 
    my_data = f.read() 
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['name'],'domain': {'id': credentials['domain']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', local_file_name])
    s_subject_token = resp1.headers['x-subject-token']
    
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'} 
    resp2 = requests.put(url=url2, headers=headers2, data = my_data) 
    print (resp2)

In [14]:
df_data_1.to_csv('wine-data-transformed.csv', index=False)

In [15]:
put_file('CourseraIoTProject', 'wine-data-transformed.csv', credentials)

<Response [201]>
